In [127]:
from __future__ import division
import pandas as pd
import numpy as np
from scipy.spatial.distance import cdist 

import warnings
warnings.filterwarnings("ignore")
from bokeh.layouts import layout,column,row
from bokeh.plotting import figure, show, output_file
from bokeh.models import ColumnDataSource

In [128]:
from bokeh.io import output_notebook, push_notebook
output_notebook()

Loading BokehJS ...

In [77]:
df=pd.read_csv('./Datasets/data_banknote_authentication.txt',header=None)
df.columns=['variance','skewness','curtosis','entropy','target']

real_data=df[df['target']==0]
fake_data=df[df['target']==1]
#real_data - assume to be actual data.
#fake data - assume to be outliers.
print("Num +ve instances:",real_data.shape)
print("Num -ve instances:",fake_data.shape)

Num +ve instances: (762, 5)
Num -ve instances: (610, 5)


In [79]:
#sample -ve to 60.
#pick every 10th point.
fake_data=fake_data.iloc[::10,:]
print("Reduced number of instances of fake_data:",fake_data.shape)

Reduced number of instances of fake_data: (61, 5)


In [80]:
#get the t-sne for the original data,actual target and store in CDS for bokeh.
original_data=pd.concat([real_data,fake_data],axis=0)
print("Final data shape",original_data.shape)

#convert it into dataset which is a numpy matrix.
dataset=original_data.as_matrix()
print("Dataset shape:",dataset.shape)

#get features as the first 4 columns and target as the last column.
features=dataset[:,:4]
labels=dataset[:,-1]
print("features shape:",features.shape,labels.shape)

Final data shape (823, 5)
Dataset shape: (823, 5)
features shape: (823, 4) (823,)


In [88]:
def get_num_neighbors(dist_list,radius=5):
    dist_=[False]*len(dist_list)
    dist_=[True for l in dist_list if l<=radius]
    return sum(dist_)

def LOC_Score(point,data):
    #compute loc score of a point with all other points.
    #use euclidean distance and loc_score=(num points within a radius r=5)/total points
    #the more dense neighborhood - higher score. less likely to be an outlier.
    dist_mat=cdist(point,data,'euclidean')
    dist_mat=sorted(dist_mat.tolist()[0])
    total_neigh=len(dist_mat)
    num_neigh=get_num_neighbors(dist_mat,5)
    score=num_neigh/total_neigh
    #print("Num neighbors within r=5",num_neigh)
    #print("Score of the point:",score)
    return score

def get_loc_scores(feats): 
    #feats - features matrix with shape NxF
    loc_scores_all_features=[]
    for i in range(feats.shape[0]):
        score=LOC_Score(feats[i,:].reshape(-1,feats.shape[1]),feats)
        loc_scores_all_features.append(score) 
    return loc_scores_all_features

loc_scores_all_features=get_loc_scores(features)

In [83]:
outlier_scores=loc_scores_all_features[-61:]
real_scores=loc_scores_all_features[:-61]
print("MAx of outliers: ",max(loc_scores_all_features[-61:]))
print("Min of real points: ",min(loc_scores_all_features[:-61]))

MAx of outliers:  0.24787363304981774
Min of real points:  0.020656136087484813


In [84]:
thresholds=[0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9]

In [135]:
def classify_point(score,th):
    #for different thresholds of scores - give differnt results
    #false negative rate vs false positive rate
    if(score<=th):
        return 1 #outlier
    else:
        return 0 #real point

def get_pred(scores,th):
    #scores - scores matrix with the predicted LOC scores.
    #th - to use as guide for classification.
    pred_=[]
    for score in scores:
        pred_.append(classify_point(score,th)) 
    return pred_

def get_predictions(scores):
    preds=[[] for _ in range(len(thresholds))]
    for i,th in enumerate(thresholds):
        preds[i]= get_pred(scores,th)
    return preds    

#preds - list of lists with predictions for different thresholds.
preds=get_predictions(loc_scores_all_features)       


In [129]:
def calculate_TP(y_true,y_pred):
    #compute with respect to ouliers. - class 1
    #num_true==1,num_pred==1 and (num_pred==num_true)
    num_true=(y_true==1)
    num_pred=(np.array(y_pred)==1)
    #print(num_true,num_pred)
    return sum(num_true==num_pred)

def calculate_FP(y_true,y_pred):
    #compute with respect to ouliers. - class 1
    #num_true==0,num_pred==1 and (num_pred==num_true)
    num_true=(np.array(y_true)==0)
    num_pred=(np.array(y_pred)==1)
    #print(num_true,num_pred)
    return sum(num_true==num_pred)

def calculate_FN(y_true,y_pred):
    #compute with respect to ouliers. - class 1
    #num_true==0,num_pred==1 and (num_pred==num_true)
    num_true=(np.array(y_true)==1)
    num_pred=(np.array(y_pred)==0)
    #print(num_true,num_pred)
    return sum(num_true==num_pred)

def calculate_TN(y_true,y_pred):
    #compute with respect to ouliers. - class 1
    #num_true==0,num_pred==1 and (num_pred==num_true)
    num_true=(np.array(y_true)==0)
    num_pred=(np.array(y_pred)==0)
    #print(num_true,num_pred)
    return sum(num_true==num_pred)

def get_rates(preds):
    detection_rate=[]
    false_rate=[]  
    for pred in preds:
        tp=calculate_TP(labels,pred)
        fn=calculate_FN(labels,pred)
        dr=tp/(tp+fn)
        detection_rate.append(dr)
    
        tn=calculate_TN(labels,pred)
        fp=calculate_FP(labels,pred)
        fr=fp/(fp+tn)
        false_rate.append(fr)
        #print(tp,fn,dr,tn,fp,fr)
    return (detection_rate,false_rate)

detection_rate,false_rate=get_rates(preds)

In [140]:
def get_Nt_Features():
    nt=4
    global features
    total=range(features.shape[1]) #5
    Nt=np.random.choice(total,nt,replace=False)   
    return Nt

def get_Ft_Features(Nt):
    #random within Nt - without replacement.
    ft=3
    return np.random.choice(Nt,ft,replace=False)

def cumsum(scores_mat):
    #scores_mat - np.array of shape TxN where
    #T- Num of algorithms
    #N- number of data points.
    return np.mean(scores_mat,axis=0)

def breadth_first(scores_mat,idxs_mat):
    ranked_prob=[]
    ranked_score=[]
    #scores_mat - np.array of shape TxN where sorted scores.
    #idxs_mat - np.array of hape TxN with ids of max score.
    for t in range(scores_mat.shape[0]):
        for m in range(scores_mat.shape[1]):
            if(idxs_mat[t,m] not in ranked_prob):
                ranked_prob.append(idxs_mat[t,m])
                ranked_score.append(scores_mat[t,m])
                
    return (ranked_prob,ranked_score)            
            

#T- number of Outlier detection algorithms.
T=10
chosen_features=[[] for _ in range(T)]
algo_loc_scores=[[] for _ in range(T)]
for t in range(T):
    Nt=get_Nt_Features()
    Ft=get_Ft_Features(Nt)
    chosen_features[t]=Ft
    feats=features[:,Ft]
    algo_loc_scores[t]=get_loc_scores(feats)

(10, 823)

In [120]:
algo_loc_scores=np.array(algo_loc_scores)
def sort_scores(scores):
    sorted_scores=scores
    sorted_idxs=list(range(len(scores)))
    #insertion sort - like playing cards.
    for i in range(len(scores)):
        max_=scores[i]
        for j in range(i+1,len(scores)):
            if(scores[j]> max_):
                sorted_scores[i]=sorted_scores[j]
                sorted_scores[j]=max_
                max_=sorted_scores[i]
                
                tmp=sorted_idxs[j]
                sorted_idxs[j]=sorted_idxs[i]
                sorted_idxs[i]=tmp 
                
                
    return (sorted_scores,sorted_idxs)           
        
sorted_algo_loc_scores=[]
sorted_algo_loc_idxs=[]
for scores in algo_loc_scores:
    sorted_scores,sorted_idxs=sort_scores(scores)
    sorted_algo_loc_scores.append(sorted_scores)
    sorted_algo_loc_idxs.append(sorted_idxs)    

In [124]:
sorted_algo_loc_scores=np.array(sorted_algo_loc_scores)
sorted_algo_loc_idxs=np.array(sorted_algo_loc_idxs)
print(sorted_algo_loc_scores.shape,sorted_algo_loc_idxs.shape)

(10, 823) (10, 823)


In [142]:
cumsum_scores=cumsum(algo_loc_scores)
ranked_prob,ranked_score=breadth_first(sorted_algo_loc_scores,sorted_algo_loc_idxs)

def rearrange_scores(idxs,scores):
    rearranged=[0]*len(idxs)
    for n_i,i in enumerate(idxs):
        rearranged[n_i]=scores[idxs[i]]
    return rearranged

bf_scores=rearrange_scores(ranked_prob,ranked_score)

preds_cumsum=get_predictions(cumsum_scores)
det_rate_cumsum,false_rate_cumsum=get_rates(preds_cumsum)

preds_bf=get_predictions(bf_scores)
det_rate_bf,false_rate_bf=get_rates(preds_bf)

In [143]:
source=ColumnDataSource(data=dict(dr=detection_rate,fr=false_rate,dr_c=det_rate_cumsum,fr_c=false_rate_cumsum,\
                       dr_bf=det_rate_bf,fr_bf=false_rate_bf))

plot=figure(plot_width=500,plot_height=400,x_axis_label='False Rate',y_axis_label='Detection Rate',title='ROC curve')
plot.line('fr','dr',source=source,line_color='red')
plot.line('fr_c','dr_c',source=source,line_color='blue')
plot.line('fr_bf','dr_bf',source=source,line_color='lightblue')
show(plot)

In [149]:
source2=ColumnDataSource(data=dict(idxs=list(range(len(preds[0]))),preds=preds[0],preds_c=preds_cumsum[0],preds_bf=preds_bf[0]))
plot=figure(plot_width=500,plot_height=400)
plot.circle('idxs','preds',source=source2)

plot2=figure(plot_width=500,plot_height=400)
plot2.circle('idxs','preds_c',source=source2)

plot3=figure(plot_width=500,plot_height=400)
plot3.circle('idxs','preds_bf',source=source2)

show(row(plot,plot2,plot3))